<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/05%20-%20TensorBoard%2C%20Saver/02_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연동을 위한 import
from google.colab import drive

# 구글 드라이브 연동을 위한 인증
drive.mount('/content/drive')

basic_path = 'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 개요

텐서보드를 이용하기 위해 각종 변수들을 설정하고 저장하는 방법을 익히기

---

+ 그런데.. colab은 가상머신을 사용하므로 tensorboard를 사용할 때 추가적인 api나 서비스가 필수적이다.

+ 슬프게도 1.x버전에서는 ngrok라는 서비스가 종료하면서  tensorboard의 사용할 수가 없게 되었다.

+ 2.x버전에서는 가능하니 꼭 tensorboard가 활용되는 것을 보고 싶다면 해당 코드의 2.1버전을 보도록 하자.

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

# TensorBoard notebook extension을 불러온다.
%load_ext tensorboard

## 데이터 정의

In [ ]:
data = np.loadtxt(basic_path + '/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

## 모델 구성

### 변수 정의

In [ ]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

### 신경망 정의

In [ ]:
# 텐서보드에서 with tf.name_scope 으로 묶은 블럭은 한 레이어안에 표현된다.
with tf.name_scope('layer1'):
  W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
  L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
  W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
  L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
  W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
  model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
  cost = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

  optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
  train_op = optimizer.minimize(cost, global_step=global_step)

  # tf.summary.scalar 를 이용해 텐서보드로 확인하고 싶은 값들을 지정할 수 있다.
  tf.summary.scalar('cost', cost)

### 모델 불러오기

In [ ]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(basic_path + '/board_model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver/board_model/dnn.ckpt-400


### 텐서보드에 사용할 그래프와 텐서값들을 저장

만약 가상머신이 아니라면 아래와 같은 방법으로 텐서보드를 실행할 수 있다.

1. 아래의 코드를 실행한다.
2. 학습 후 다음의 명령어를 이용해 웹서버를 실행시킨 뒤 \
tensorboard --logdir=./logs \
위의 명령어를 cmd명령창에서 실행시킨다.
3. 웹 브라우저에서 다음의 주소를 넣어서 텐서보드에서 확인할 수 있다.\
http://localhost:6006

In [ ]:
# 텐서보드에서 표시해주기 위한 텐서들을 수집한다.
merged = tf.summary.merge_all()
# 위의 텐서값들을 어디에 저장할지 설정한다.
writer = tf.summary.FileWriter(basic_path + '/logs', sess.graph)

### 모델 학습 및 저장

In [ ]:
for step in range(100):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

  # 적절한 시점에 저장할 값들을 수집하고 저장한다.
  # 이 코드에 경우에는 매 훈련마다 저장하고 있다.
  summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
  writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, basic_path + '/board_model/dnn.ckpt', global_step=global_step)

Step: 401,  Cost: 0.549
Step: 402,  Cost: 0.549
Step: 403,  Cost: 0.549
Step: 404,  Cost: 0.549
Step: 405,  Cost: 0.549
Step: 406,  Cost: 0.549
Step: 407,  Cost: 0.549
Step: 408,  Cost: 0.549
Step: 409,  Cost: 0.549
Step: 410,  Cost: 0.549
Step: 411,  Cost: 0.549
Step: 412,  Cost: 0.549
Step: 413,  Cost: 0.549
Step: 414,  Cost: 0.549
Step: 415,  Cost: 0.549
Step: 416,  Cost: 0.549
Step: 417,  Cost: 0.549
Step: 418,  Cost: 0.549
Step: 419,  Cost: 0.549
Step: 420,  Cost: 0.549
Step: 421,  Cost: 0.549
Step: 422,  Cost: 0.549
Step: 423,  Cost: 0.549
Step: 424,  Cost: 0.549
Step: 425,  Cost: 0.549
Step: 426,  Cost: 0.549
Step: 427,  Cost: 0.549
Step: 428,  Cost: 0.549
Step: 429,  Cost: 0.549
Step: 430,  Cost: 0.549
Step: 431,  Cost: 0.549
Step: 432,  Cost: 0.549
Step: 433,  Cost: 0.549
Step: 434,  Cost: 0.549
Step: 435,  Cost: 0.549
Step: 436,  Cost: 0.549
Step: 437,  Cost: 0.549
Step: 438,  Cost: 0.549
Step: 439,  Cost: 0.549
Step: 440,  Cost: 0.549
Step: 441,  Cost: 0.549
Step: 442,  Cost

'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver/board_model/dnn.ckpt-500'

###모델 테스트

In [ ]:
# 0: 기타 1: 포유류, 2: 조류

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


# 전체 코드

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

data = np.loadtxt(basic_path + '/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#########
# 신경망 모델 구성
######

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost)


#########
# 신경망 모델 학습
######

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(basic_path + '/board_model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(basic_path + '/logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, basic_path + '/board_model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

NameError: ignored